Init workspace


In [2]:
!source .venv/bin/activate

In [4]:
%pip install -qr requirements.txt clearml  # install

import torch
import utils

display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-279-gae7a799 Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)


Setup complete ✅ (12 CPUs, 15.6 GB RAM, 119.3/1006.9 GB disk)


Clone Dataset


In [ ]:
!clearml-data get \
--id 63c8e9373cd44d278ebfe693a575387f \
--copy ../datasets/yolo \
--overwrite

Train


In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/test \
--name train_bs32_ep75 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 75 \
--img 736 \
--cache

In [ ]:
!python train.py \
--project YOLOv5-Pill-Counter/Evolution \
--name train_bs32_ep30 \
--weights "" \
--cfg yolov5s.yaml \
--data yolo.yaml \
--batch-size 32 \
--epochs 30 \
--img 736 \
--cache \
--evolve \
--resume

Export


In [3]:
!python export.py \
--weights YOLOv5-Pill-Counter/test/train_50/weights/best.pt \
--include tfjs \
--imgsz 736

/home/czack913/Code/yolov5/export.py:59: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
export: data=data/coco128.yaml, weights=['YOLOv5-Pill-Counter/test/train_50/weights/best.pt'], imgsz=[736], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['tfjs']
YOLOv5 🚀 v7.0-279-gae7a799 Python-3.10.12 torch-2.1.2+cu121 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 70

Validate


In [ ]:
!python val.py \
--weights runs/train/exp2/weights/best.pt \
--data yolo.yaml \
--img 736

Detect


In [ ]:
!python detect.py \
--weights runs/train/exp2/weights/best.pt \
--source yolo.yaml \
--img 736